In [1]:
import numpy as np
import pandas as pd
import pymannkendall as mk
import os, glob
from pathlib import Path
from matplotlib import pyplot
import datetime
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt

In [2]:
g = os.getcwd()
g
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

# load smooth data

In [3]:
alldf = pd.read_csv("Data_IDB/alldf_smooth.csv")
type(alldf.date[0])
alldf.index = pd.to_datetime(alldf['date'])
alldf['date'] = pd.to_datetime(alldf['date'])
# alldf

- mean moth by ID

In [4]:
#create deseason data
deseason_trends = pd.DataFrame() 
group_id= alldf.groupby(["ID"])
for fid in alldf.ID.unique():
#     print(fid)
    ID = group_id.get_group(fid)
    ID
    monthly_mean = ID.resample('M').mean()
    # monthly_mean.shift()
    # ############deseason
#     monthly_mean_diff = monthly_mean - monthly_mean.shift()
#     monthly_mean_diff["ID"] = monthly_mean["ID"]
    deseason_trends = pd.concat([deseason_trends,monthly_mean],axis=0)#
    deseason_trends.dropna(inplace=True)
# deseason_trends

In [5]:
deseason_trends.to_csv("Data_IDB/data_monthly.csv")
# deseason_trends

- filter by year <=2014

In [35]:
df_2014 = deseason_trends[deseason_trends.index<"01-01-2015"]

In [37]:
groups = df_2014.groupby(["ID"],as_index=False)
# groups.groups.keys()
all_trends = pd.DataFrame()
for fid in alldf.ID.unique():
    db = groups.get_group(fid)
    trends_Ori = mk.original_test(db.OSAVI) #period=12 
    trends_Mod_rao = mk.hamed_rao_modification_test(np.array(db.OSAVI))
    trends_Mod_yue =mk.yue_wang_modification_test(np.array(db.OSAVI))
    trends_seasonal =mk.seasonal_test(np.array(db.OSAVI),period = 12)
#     cor_seasonal =mk.correlated_seasonal_test(np.array(db.OSAVI),period = 12)
    row = pd.DataFrame({'ID':[fid],'trend_Ori': [trends_Ori[0]],'Pvalue_Ori':[trends_Ori[2]],
                        'trends_Mod_yue': [trends_Mod_yue[0]],'Pvalue_Mod_yue':[trends_Mod_yue[2]],
                        'trends_Mod_rao': [trends_Mod_rao[0]],'Pvalue_Mod_rao':[trends_Mod_rao[2]],
                        'trends_seasonal': [trends_seasonal[0]],'Pvalue_seasonal':[trends_seasonal[2]],
                        'cor_seasonal': [cor_seasonal[0]],'Pvalue_seasonal_cor':[cor_seasonal[2]],
                        'test': ['mandal kendal']})
    all_trends = pd.concat([all_trends,row],ignore_index=True)

all_trends.to_csv("Data_IDB/deseason_kendal_osavi_2014.csv")

C:\Users\fmontenegro\Miniconda3\envs\GEE\lib\site-packages\pymannkendall\pymannkendall.py:103: RuntimeWarning: invalid value encountered in sqrt
  z = (s + 1)/np.sqrt(var_s)


# by YEAR

In [ ]:
years = ['2012-12-31','2013-12-31','2014-12-31','2015-12-31','2016-12-31','2017-12-31','2018-12-31','2019-12-31']
all_trends = pd.DataFrame()
for fid in alldf.ID.unique():
    db = groups.get_group(fid)
    all_row = pd.DataFrame()
    for year in years:
        df_year = db[db.date<=year]
        trends_by_year_Ori = mk.original_test(df_year.NDVI) #period=12 
        trends_by_year_Mod = mk.yue_wang_modification_test(db.NDVI)
        row = pd.DataFrame({'trend_Ori'+year[0:4]: [trends_by_year_Ori[0]],'trend_Mod'+year[0:4]: [trends_by_year_Mod[0]]})
        all_row = pd.concat([all_row,row],axis = 1)
    all_row['ID'] = fid
    all_trends = pd.concat([all_trends,all_row],ignore_index=True)

In [ ]:
years = ['2012-12-31','2013-12-31','2014-12-31','2015-12-31','2016-12-31','2017-12-31','2018-12-31','2019-12-31']
years[0][0:4]

In [ ]:
all_row

In [ ]:
all_trends['test'] = "mandal kendal"

In [ ]:
all_trends

- test No trend

In [ ]:
all_trends[all_trends.ID==386511]

In [ ]:
all_trends[all_trends.ID==396011]